In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('base_data.csv')
df.head()

,AGREEMENT_RK,ID_CLIENT,TARGET,ID,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,...,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,FL_PRESENCE_FL,OWN_AUTO,FAMILY_INCOME,PERSONAL_INCOME,ID_LOAN,CLOSED_FL
0,60099204,106805103,1,106805103,42,1,Среднее,Не состоял в браке,1,0,...,0,Московская область,Московская область,Московская область,1,0,от 20000 до 50000 руб.,25000.0,1753791446,0
1,62244665,106809308,0,106809308,28,1,Среднее специальное,Состою в браке,1,1,...,0,Читинская область,Читинская область,Читинская область,0,0,от 10000 до 20000 руб.,10000.0,1753796120,0
2,61050759,106805867,0,106805867,64,0,Среднее специальное,Состою в браке,2,0,...,1,Иркутская область,Иркутская область,Иркутская область,0,1,от 20000 до 50000 руб.,30000.0,1753792244,1
3,62079659,106808779,0,106808779,54,1,Среднее специальное,Состою в браке,0,0,...,0,Новосибирская область,Новосибирская область,Новосибирская область,1,1,от 20000 до 50000 руб.,25000.0,1753795547,0
4,66583553,106814289,0,106814289,26,0,Среднее специальное,Состою в браке,1,1,...,0,Красноярский край,Красноярский край,Красноярский край,1,0,от 10000 до 20000 руб.,15000.0,1753802108,1


In [3]:
print(df.isna().sum(), '\n-----------\n', df.isnull().sum())

AGREEMENT_RK               0
ID_CLIENT                  0
TARGET                     0
ID                         0
AGE                        0
GENDER                     0
EDUCATION                  0
MARITAL_STATUS             0
CHILD_TOTAL                0
DEPENDANTS                 0
SOCSTATUS_WORK_FL          0
SOCSTATUS_PENS_FL          0
REG_ADDRESS_PROVINCE       0
FACT_ADDRESS_PROVINCE      0
POSTAL_ADDRESS_PROVINCE    0
FL_PRESENCE_FL             0
OWN_AUTO                   0
FAMILY_INCOME              0
PERSONAL_INCOME            0
ID_LOAN                    0
CLOSED_FL                  0
dtype: int64 
-----------
 AGREEMENT_RK               0
ID_CLIENT                  0
TARGET                     0
ID                         0
AGE                        0
GENDER                     0
EDUCATION                  0
MARITAL_STATUS             0
CHILD_TOTAL                0
DEPENDANTS                 0
SOCSTATUS_WORK_FL          0
SOCSTATUS_PENS_FL          0
REG_ADDRESS_PROV

По выводу df.isna, df.isnull видим, что пропущенных значений нет (надеюсь что запросы в sql были написаны корректно, и нам просто повезло с этим)

In [4]:
loan_closed = df.groupby('ID_CLIENT')['CLOSED_FL'].apply(lambda x: (x == 1).sum()).reset_index(name='LOAN_NUM_CLOSED')
loan_total = df.groupby('ID_CLIENT')['ID_LOAN'].count().reset_index(name='LOAN_NUM_TOTAL')

df = df.merge(loan_closed, how='outer')
df = df.merge(loan_total, how='outer')
df.head()

,AGREEMENT_RK,ID_CLIENT,TARGET,ID,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,...,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,FL_PRESENCE_FL,OWN_AUTO,FAMILY_INCOME,PERSONAL_INCOME,ID_LOAN,CLOSED_FL,LOAN_NUM_CLOSED,LOAN_NUM_TOTAL
0,60099204,106805103,1,106805103,42,1,Среднее,Не состоял в браке,1,0,...,Московская область,Московская область,1,0,от 20000 до 50000 руб.,25000.0,1753791446,0,0,1
1,62244665,106809308,0,106809308,28,1,Среднее специальное,Состою в браке,1,1,...,Читинская область,Читинская область,0,0,от 10000 до 20000 руб.,10000.0,1753796120,0,0,1
2,61050759,106805867,0,106805867,64,0,Среднее специальное,Состою в браке,2,0,...,Иркутская область,Иркутская область,0,1,от 20000 до 50000 руб.,30000.0,1753792244,1,1,1
3,62079659,106808779,0,106808779,54,1,Среднее специальное,Состою в браке,0,0,...,Новосибирская область,Новосибирская область,1,1,от 20000 до 50000 руб.,25000.0,1753795547,0,0,1
4,66583553,106814289,0,106814289,26,0,Среднее специальное,Состою в браке,1,1,...,Красноярский край,Красноярский край,1,0,от 10000 до 20000 руб.,15000.0,1753802108,1,2,2


In [5]:
# оставляем только нужные нам колонки (пока это все числовые признаки)
df = df[
    [
        'AGREEMENT_RK',
        'TARGET',
        'AGE',
        'GENDER',
        'SOCSTATUS_WORK_FL',
        'SOCSTATUS_PENS_FL',
        'CHILD_TOTAL',
        'DEPENDANTS',
        'PERSONAL_INCOME',
        'LOAN_NUM_TOTAL',
        'LOAN_NUM_CLOSED',
        'FL_PRESENCE_FL',
        'OWN_AUTO',
    ]
  ]
# сейчас есть дубликаты по клиентам
# убираем их (ожидаем кол-во записей равное кол-ву в таблице D_target - 15223)
print(df.duplicated(keep=False).sum())
df = df.drop_duplicates()
df.info()

10477
<class 'pandas.core.frame.DataFrame'>
Int64Index: 15223 entries, 0 to 21534
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AGREEMENT_RK       15223 non-null  int64  
 1   TARGET             15223 non-null  int64  
 2   AGE                15223 non-null  int64  
 3   GENDER             15223 non-null  int64  
 4   SOCSTATUS_WORK_FL  15223 non-null  int64  
 5   SOCSTATUS_PENS_FL  15223 non-null  int64  
 6   CHILD_TOTAL        15223 non-null  int64  
 7   DEPENDANTS         15223 non-null  int64  
 8   PERSONAL_INCOME    15223 non-null  float64
 9   LOAN_NUM_TOTAL     15223 non-null  int64  
 10  LOAN_NUM_CLOSED    15223 non-null  int64  
 11  FL_PRESENCE_FL     15223 non-null  int64  
 12  OWN_AUTO           15223 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 1.6 MB


In [6]:
df.describe()

,AGREEMENT_RK,TARGET,AGE,GENDER,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,FL_PRESENCE_FL,OWN_AUTO
count,1.522300e+04,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000
mean,6.540183e+07,0.119030,40.406096,0.654536,0.909610,0.134468,1.099389,0.645208,13853.836323,1.414636,0.766406,0.309794,0.116337
std,4.568181e+06,0.323835,11.601068,0.475535,0.286748,0.341165,0.995411,0.812252,9015.467617,0.839538,1.024446,0.462424,0.320844
min,5.991015e+07,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,1.000000,0.000000,0.000000,0.000000
25%,6.192092e+07,0.000000,30.000000,0.000000,1.000000,0.000000,0.000000,0.000000,8000.000000,1.000000,0.000000,0.000000,0.000000
50%,6.437100e+07,0.000000,39.000000,1.000000,1.000000,0.000000,1.000000,0.000000,12000.000000,1.000000,0.000000,0.000000,0.000000
75%,6.708802e+07,0.000000,50.000000,1.000000,1.000000,0.000000,2.000000,1.000000,17000.000000,2.000000,1.000000,1.000000,0.000000
max,7.529224e+07,1.000000,67.000000,1.000000,1.000000,1.000000,10.000000,7.000000,250000.000000,16.000000,16.000000,1.000000,2.000000


In [7]:
df.isna().sum()

AGREEMENT_RK         0
TARGET               0
AGE                  0
GENDER               0
SOCSTATUS_WORK_FL    0
SOCSTATUS_PENS_FL    0
CHILD_TOTAL          0
DEPENDANTS           0
PERSONAL_INCOME      0
LOAN_NUM_TOTAL       0
LOAN_NUM_CLOSED      0
FL_PRESENCE_FL       0
OWN_AUTO             0
dtype: int64

In [9]:
# проверяем что все agreement_pk уникальны
assert len(df['AGREEMENT_RK'].unique()) == df.shape[0]

На этом обработку датасета завершаем

In [10]:
df.to_csv('data.csv', index=False)